# **<center> Análisis de Sentimientos de los candidatos presidenciales </br> Ecuador 2021, en la red social Twitter </center>**
<h6>
    <center> Miguel Angel Macias, Jonnathan Campoberde <br/> 
    <i>Universidad de Cuenca <br/>  <i>Facultad de Ingenieria - Escuela de Sistemas<br/> Cuenca, Ecuador</i>
    <br/> { mangel.maciasn, jonnathan.campoberde }@ucuenca.edu.ec</center>
<h6>

# **I.&emsp;Introducción**
El presente trabajo tiene como objetivo emplear de manera práctica todos los conocimientos adquiridos en el transcurso de la asignatura “Text Mining”. En este trabajo se realizó una limpieza de los datos, se removieron caracteres especiales, emojis y enlaces de los tweets recolectados; así también se realizó una inspección de posibles cuentas trolls que afecten el AS de las opiniones de usuarios a favor o en contra de un movimiento político. Luego del tratamiento y análisis de los tweets se entrenaron 3 modelos de clasificación de texto usando Support Vector Machine (SVM), Naive Bayes (NB) y Random Forest (RF) para posteriormente compararlos y elegir qué modelo presentó los mejores resultados. 
Los tweets que se recolectaron tratan acerca de los dos candidatos presidenciales que lideran las encuestas para las elecciones de Ecuador 2021. Los dos candidatos, Andrés Arauz y Guillermo Lasso, pertenecen a los dos movimientos políticos más dominantes del estado ecuatoriano.

## **1.1.&emsp;Objetivos del Proyecto**
>. El objetivo de este artículo es el AS de estos dos candidatos para luego contrastarlos con los resultados oficiales de las elecciones, así también, entrenar un modelo de clasificación de sentimientos de los tweets que se generen en redes sociales acerca de la política dentro del país. Posteriormente se desea responder a las siguientes preguntas de investigación definidas en 1.2.

## **1.2.&emsp;Preguntas de investigación**
>- RQ1:     ¿Qué tan preciso los datos de Twitter reflejan el sentimiento político en un proceso electoral?
>- RQ1:     ¿En qué grado de aceptación se encuentran los candidatos dentro de la red social Twitter?
>- RQ3:     ¿Los resultados oficiales son parecidos a los resultados encontrados en nuestro análisis? 


# **II.&emsp;TEXT CLEANING**

## **2.1.&emsp; Reading dataset**
>Para este paso...

In [9]:
#!pip install emoji

In [10]:
import warnings
warnings.filterwarnings("ignore")

#from datetime import datetime, timezone
import pytz
import numpy as np              # Para operaciones matematicas
import pandas as pd             # Manejo de datos / dataframes
import re
import emoji
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.stem import PorterStemmer
%matplotlib inline

In [11]:
data_raw = '../data/raw'              # directorio que contiene toda la data sin manipular
data_processed = '../data/processed'  # directorio que contiene data procesada

In [13]:
df_arauz = pd.read_csv(data_raw+"/tweets_about_arauz.csv")
df_arauz['created_at'] = pd.to_datetime(df_arauz['created_at'], utc=pytz.UTC)
df_arauz['user.created_at'] = pd.to_datetime(df_arauz['user.created_at'], utc=pytz.UTC)
df_arauz

,created_at,favorite_count,full_text,id_str,metadata.iso_language_code,retweeted_status.created_at,retweeted_status.full_text,user.created_at,user.favourites_count,user.followers_count,user.friends_count,user.id_str,user.location,user.verified,retweet_count
0,2021-01-24 11:09:43+00:00,0,RT @CarlosP05340289: El LELO Andrés Arauz nos ...,1353298931913019393,es,Sat Jan 23 22:05:45 +0000 2021,El LELO Andrés Arauz nos ha dicho q ha gestion...,2019-07-08 04:50:16+00:00,23792,208,814,1148091911183503360,NaN,False,212
1,2021-01-24 11:11:10+00:00,0,RT @sybelmartinez: A qué edad se enteraron que...,1353299298407096320,es,Sat Jan 23 23:38:46 +0000 2021,A qué edad se enteraron que Jorge Glas y André...,2012-10-09 14:57:55+00:00,1835,88,325,870030324,NaN,False,545
2,2021-01-24 11:11:47+00:00,0,RT @veronicaariasf: Con una extraordinaria car...,1353299454103838720,es,Sun Jan 24 00:05:24 +0000 2021,"Con una extraordinaria caravana, #Loja recibió...",2019-12-07 21:33:55+00:00,20154,73,45,1203427383782629376,NaN,False,466
3,2021-01-24 11:12:50+00:00,0,"RT @kolectiVOZ: David Villamar: ""Los #MilDeUna...",1353299717485158400,es,Sun Jan 24 01:30:39 +0000 2021,"David Villamar: ""Los #MilDeUna, propuesta de A...",2013-08-06 16:33:44+00:00,78836,1008,1396,1650738853,Un Mundo donde haya Justicia,False,292
4,2021-01-24 11:14:35+00:00,0,RT @PrensaDelBanano: Andrés Arauz fue el ganad...,1353300160277852160,es,Sat Jan 23 16:41:37 +0000 2021,Andrés Arauz fue el ganador del #DebatePreside...,2019-08-29 23:23:54+00:00,81000,290,437,1167216174972702724,NaN,False,116
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202315,2021-02-13 23:59:52+00:00,0,RT @VickyDavilaH: SEMANA: Samper está furioso ...,1360740507043966979,es,Sat Feb 13 20:21:35 +0000 2021,SEMANA: Samper está furioso y defiende al cand...,2010-04-28 02:20:18+00:00,126408,384,163,137895474,NaN,False,474
202316,2021-02-13 23:59:53+00:00,2,@tioelmoi Diana Salazar no debias meterte a tr...,1360740510298734592,es,NaN,NaN,2013-06-30 16:33:16+00:00,10,78,72,1558430348,Ecuador,False,0
202317,2021-02-13 23:59:53+00:00,0,"RT @farriancito: Andres Venegas Londoño, alias...",1360740510890160129,es,Sat Feb 13 23:56:29 +0000 2021,"Andres Venegas Londoño, alias 'Uriel', fue dad...",2019-11-12 17:48:40+00:00,21782,135,1138,1194311000910188550,NaN,False,55
202318,2021-02-13 23:59:56+00:00,0,RT @PatricioMery: El escenario ideal para el E...,1360740523892473859,es,Sat Feb 13 18:50:33 +0000 2021,El escenario ideal para el Estado Profundo: in...,2010-05-07 23:35:40+00:00,6949,4479,5000,141383221,Ecuador - Guayaco,False,173


## **2.2.&emsp; Removing Duplicates**
>lorem ipsum

In [14]:
# Count duplicates
len(df_arauz['full_text'])-len(df_arauz['full_text'].drop_duplicates())

180960

In [15]:
df_arauz_processed = df_arauz.drop_duplicates( ['full_text'], keep='first')

In [16]:
len(df_arauz_processed['full_text'])

21360

## **2.3.&emsp; Identify tweets by trolls**
>Lorem ipsum

In [17]:
def is_troll(Dt, Dc, Fc, fc, v):
    '''
    ----------------------------------
    | Variable Description           |
    ----------------------------------
    | Dt  stands for Date Tweet      |
    | Dc  stands for Date creation   |
    | Fc  stands for Followers count |
    | fc  stands for friend count    |
    | v   stands for verified        |
    ----------------------------------
    | Weight penalization            |
    ----------------------------------
    | Timeliness metric:  WD = 0.7   |
    | Followers metric:   WF = 0.2   |
    | Following metric:   Wf = 0.1   |
    ----------------------------------
    | Acceptance Threshold           |
    ----------------------------------
    | Non Troll   > 0.8              |
    | Troll       < 0.8              |
    ----------------------------------
    '''
    # If user is verified then is a real person
    if v:
        return True
    else:

        threshold = 0.7*((Dt-Dc).days)/180 + 0.2*(Fc/250) + 0.1*(fc/300)

        return threshold < 0.8


In [18]:
df_arauz_processed['is_troll'] = df_arauz_processed.apply(lambda x: is_troll(x['created_at'],
                                                         x['user.created_at'],
                                                         x['user.followers_count'],
                                                         x['user.friends_count'],
                                                         x['user.verified']), axis=1)

## **2.4.&emsp; Removing characters and Specific Stopwords**
>lorem ipsum

In [19]:
def clean_text(text):
    """Function to clean text extracted from the Internet with html tags
    
    Parameters
    ----------
    text: str 
      original string
    
    Return
    ------
    text: str
      modified initial string
    """

    text = text.lower() # lowercase text
    text = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", text) #Remove http links
    text = EMOJI.sub(r'', text)
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text

In [20]:
# filtrado de caracteres y stopwords
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@#:,;¿?!¡-]')
EMOJI = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)

STOPWORDS = ('rt')  # Adding new stopwords to the set, 'cause theese words repeat in all docs

df_arauz_processed['full_text'] = df_arauz_processed["full_text"].apply(clean_text)

### _Luego de remover los caracteres especiales pueden existir Tweets vacios, debido a que estos solo contenian emotes o enlaces en su interior. Es por esta razon que se buscan estos espacios en blanco y se los quita_

In [21]:
df_arauz_processed[df_arauz_processed['full_text']==""]["full_text"].count()

48

In [22]:
df_arauz_processed.drop(df_arauz_processed.loc[df_arauz_processed["full_text"]==""].index, inplace=True)

In [23]:
df_arauz_processed[df_arauz_processed['full_text']==""]["full_text"].count()

0

In [24]:
df_arauz_processed.to_csv(data_processed+'/arauz_clean.csv', 
                        index=False, header=True, chunksize=10000)